<a href="https://colab.research.google.com/github/Eshita781996/SCD/blob/main/SCD2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
!pip -q install findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages io.delta:delta-core_2.12:0.7.0 --conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog pyspark-shell'

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('delta_session').getOrCreate()

In [ ]:
# employees_df = spark.createDataFrame([(1, 'John', 'Sydney', 35000.00),(2, 'Peter', 'Melbourne', 45000.00),(3, 'Sam', 'Sydney',55000.00)],
#                                      schema='emp_id int,emp_name string,emp_city string,emp_salary float')

employees_delta_df = spark.createDataFrame([(4, 'Max', 'Melbourne', 55001.00),(5, 'Jessie', 'Brisbane', 48001.00)],
                                           schema='emp_id int,emp_name string,emp_city string,emp_salary float')

In [ ]:
employees_delta_df.createOrReplaceTempView('INPUT_VIEW')

In [ ]:
employees_delta_df.show()

+------+--------+---------+----------+
|emp_id|emp_name| emp_city|emp_salary|
+------+--------+---------+----------+
|     4|     Max|Melbourne|   55001.0|
|     5|  Jessie| Brisbane|   48001.0|
+------+--------+---------+----------+



In [ ]:
#For the first time
from delta.tables import *
if not DeltaTable.isDeltaTable(spark,'employees'):
  employees_delta_df.createOrReplaceTempView('v')
  df = spark.sql('select *,from_utc_timestamp(current_timestamp(),"IST") as dw_insert_ts,from_utc_timestamp(current_timestamp(),"IST") as dw_update_ts,"Y" as active_flag from v')
  df.write.format("delta").save("employees")
else:
  df = spark.read.format("delta").load("employees")
  df.createOrReplaceTempView('employees')
  DF_STAGED_UPDATE = spark.sql(f"merge into employees using ( select INPUT_VIEW.emp_id as mergerkey0,INPUT_VIEW.*,from_utc_timestamp(current_timestamp(),'IST') as dw_insert_ts,from_utc_timestamp(current_timestamp(),'IST') as dw_update_ts,'Y' as active_flag from INPUT_VIEW \
  union all select NULL as mergerkey0, INPUT_VIEW.*,from_utc_timestamp(current_timestamp(),'IST') as dw_insert_ts,from_utc_timestamp(current_timestamp(),'IST') as dw_update_ts,employees.active_flag from INPUT_VIEW join employees on employees.emp_id=INPUT_VIEW.emp_id where employees.active_flag ='Y' and \
  employees.emp_salary<>INPUT_VIEW.emp_salary ) staged_updates \
  on employees.emp_id = staged_updates.mergerkey0 and  employees.active_flag='Y' \
  when matched and employees.active_flag='Y' and  \
  employees.emp_salary<>staged_updates.emp_salary and employees.emp_id=staged_updates.emp_id \
  then update set active_flag='N',dw_update_ts = from_utc_timestamp(current_timestamp(),'IST') \
  when not matched then insert *")
  df = spark.sql('select * from employees')
  df.show()
  df.write.format("delta").mode("overwrite").save("employees")


+------+--------+---------+----------+--------------------+--------------------+-----------+
|emp_id|emp_name| emp_city|emp_salary|        dw_insert_ts|        dw_update_ts|active_flag|
+------+--------+---------+----------+--------------------+--------------------+-----------+
|     5|  Jessie| Brisbane|   48000.0|2024-01-04 20:02:...|2024-01-04 20:22:...|          N|
|     5|  Jessie| Brisbane|   42000.0|2024-01-04 20:02:...|2024-01-04 20:17:...|          N|
|     4|     Max|Melbourne|   55000.0|2024-01-04 20:17:...|2024-01-04 20:22:...|          N|
|     4|     Max|Melbourne|   55001.0|2024-01-04 20:22:...|2024-01-04 20:22:...|          Y|
|     5|  Jessie| Brisbane|   48001.0|2024-01-04 20:22:...|2024-01-04 20:22:...|          Y|
|     2|   Peter|Melbourne|   55000.0|2024-01-04 19:07:...|2024-01-04 19:07:...|          Y|
|     2|   Peter|Melbourne|   45000.0|2024-01-04 19:07:...|2024-01-04 20:02:...|          N|
|     1|    John|   Sydney|   35000.0|2024-01-04 19:07:...|2024-01-04 

In [ ]:
spark.read.format("delta").load("employees").show()

+------+--------+---------+----------+--------------------+--------------------+-----------+
|emp_id|emp_name| emp_city|emp_salary|        dw_insert_ts|        dw_update_ts|active_flag|
+------+--------+---------+----------+--------------------+--------------------+-----------+
|     1|    John|   Sydney|   35000.0|2024-01-04 19:07:...|2024-01-04 19:07:...|          Y|
|     3|     Sam|   Sydney|   55000.0|2024-01-04 19:07:...|2024-01-04 19:07:...|          Y|
|     5|  Jessie| Brisbane|   48000.0|2024-01-04 20:02:...|2024-01-04 20:22:...|          N|
|     5|  Jessie| Brisbane|   42000.0|2024-01-04 20:02:...|2024-01-04 20:17:...|          N|
|     4|     Max|Melbourne|   55000.0|2024-01-04 20:17:...|2024-01-04 20:22:...|          N|
|     4|     Max|Melbourne|   55001.0|2024-01-04 20:22:...|2024-01-04 20:22:...|          Y|
|     5|  Jessie| Brisbane|   48001.0|2024-01-04 20:22:...|2024-01-04 20:22:...|          Y|
|     2|   Peter|Melbourne|   55000.0|2024-01-04 19:07:...|2024-01-04 